In [1]:
### see also https://www.tensorflow.org/tfx/serving/docker
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving
Digest: sha256:df3c6fe1fbe5ccc3a916984ff313cc2d17e617f7b8782fc31e762c491325d813
Status: Image is up to date for tensorflow/serving:latest


In [ ]:
### see also https://towardsdatascience.com/deploying-keras-models-using-tensorflow-serving-and-flask-508ba00f1037

import tensorflow as tf
import os

tf.keras.backend.set_learning_phase(0) 
model = tf.keras.models.load_model('./models/xor.h5')
export_path = './models/1'


os.system("rm -rf " + export_path)

# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors
# And stored with the default serving key
with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': model.input},
        outputs={t.name: t for t in model.outputs})

In [23]:
!docker run --name serving -t --rm -p 8501:8501 \
    -v "$PWD/models:/models/xor" \
    -e MODEL_NAME=xor \
    tensorflow/serving

2019-05-14 11:22:54.086804: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: xor model_base_path: /models/xor
2019-05-14 11:22:54.087789: I tensorflow_serving/model_servers/server_core.cc:461] Adding/updating models.
2019-05-14 11:22:54.087914: I tensorflow_serving/model_servers/server_core.cc:558]  (Re-)adding model: xor
2019-05-14 11:22:54.199883: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: xor version: 1}
2019-05-14 11:22:54.199969: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: xor version: 1}
2019-05-14 11:22:54.200016: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: xor version: 1}
2019-05-14 11:22:54.200611: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim from: /models/xor/1
2019-05-14 11:22:54.2008

In [22]:
!docker stop $(docker ps -a -q --filter name=serving)
#!docker rm $(docker stop $(docker ps -a -q --filter name=serving))

9b5b995e801e
